In [ ]:
# ==== 시나리오 성능 비교 (이미지 공식: 시간 기반 우회율 적용 버전) ====
import os, json, time, math
from pathlib import Path
import pandas as pd
import requests
from collections import defaultdict

# --------- 사용자 설정 ---------
# 1~1366 순차 번호가 적용된 파케이 파일 경로
PARQUET_PATH = "../data/processed/240307_wirye_od.parquet"
# OSRM 서버 주소 (현재 실행 중인 8000번 포트 반영)
OSRM_BASE = "http://127.0.0.1:8000" 

# 분석할 시나리오 경로 설정
SCENARIOS = {
    "10대(nonkey1)": "../outputs/10대(nonkey1)",
    "20대(nonkey1)": "../outputs/20대(nonkey1)",
    "30대_nonkey1": "../outputs/30대(nonkey1)",
    "50대_nonkey1": "../outputs/50대(nonkey1)", 
}

# --------- 유틸 함수 ---------
def jload(p):
    if not os.path.exists(p): return None
    with open(p, "r", encoding="utf-8") as f:
        try: return json.load(f)
        except:
            f.seek(0); return [json.loads(line) for line in f if line.strip()]

def get_list(x):
    if isinstance(x, list): return x
    if isinstance(x, dict):
        for k in ("events","moves"):
            if isinstance(x.get(k), list): return x[k]
    return []

_OSRM_TIME_CACHE = {}
def osrm_route_duration_sec(lon1, lat1, lon2, lat2):
    """OSRM에서 최단 도로 주행 시간(초)을 가져옴"""
    if None in (lon1, lat1, lon2, lat2): return 0.0
    key = (round(float(lon1),6), round(float(lat1),6), round(float(lon2),6), round(float(lat2),6))
    if key in _OSRM_TIME_CACHE: return _OSRM_TIME_CACHE[key]
    
    url = f"{OSRM_BASE}/route/v1/driving/{lon1},{lat1};{lon2},{lat2}"
    try:
        r = requests.get(url, params={"overview":"false"}, timeout=2.0)
        if r.status_code == 200:
            data = r.json()
            if data.get("routes"):
                sec = data["routes"][0]["duration"] # OSRM 기본 단위: 초(seconds)
                _OSRM_TIME_CACHE[key] = sec
                return sec
    except: pass
    return 0.0 

# --------- ① 원본 데이터 로드 ---------
# 요청 시각과 OD 좌표를 가져와서 대기시간 및 최단시간 계산에 사용
pq = pd.read_parquet(PARQUET_PATH).copy()
pq["KEY1"] = pq["KEY1"].astype(str)
req_df = pd.DataFrame({
    "req_id": pq["KEY1"],
    "t_request": pd.to_numeric(pq["승차_timestamp"], errors="coerce"),
    "o_lon": pd.to_numeric(pq["출발_x"], errors="coerce"), "o_lat": pd.to_numeric(pq["출발_y"], errors="coerce"),
    "d_lon": pd.to_numeric(pq["도착_x"], errors="coerce"), "d_lat": pd.to_numeric(pq["도착_y"], errors="coerce")
})
req_df = req_df.dropna(subset=["t_request"])

# --------- ② 시나리오별 분석 루프 ---------
rows = []
for scen, folder in SCENARIOS.items():
    if not os.path.exists(folder): continue
    events = get_list(jload(os.path.join(folder, "events.json")))
    summary = jload(os.path.join(folder, "summary.json")) or {}

    # 1. 배차 성공 여부 및 대기시간 계산
    pick_times = {str(e["req_id"]): float(e["t"]) for e in events if str(e.get("type")).upper()=="PICKUP"}
    num_passengers = len(pick_times)
    
    pick_df = pd.DataFrame({"req_id": list(pick_times.keys()), "t_pickup": list(pick_times.values())})
    merged = req_df.merge(pick_df, on="req_id")
    # 대기시간 = 실제 승차시각 - 승차 요청시각
    merged["wait_sec"] = (merged["t_pickup"] - merged["t_request"]).clip(lower=0)
    
    # 2. 우회율 계산 (이미지 공식: DRT 탑승시간 / 최단 도로 주행 시간)
    passengers = {} # 개별 승객의 PICKUP/DROPOFF 정보 저장
    for e in events:
        etype = str(e.get("type")).upper()
        rid = str(e.get("req_id"))
        if rid == "None": continue
        if etype == "PICKUP":
            passengers[rid] = {"t_pick": float(e.get("t")), "o": (e["lon"], e["lat"])}
        elif etype == "DROPOFF" and rid in passengers:
            passengers[rid]["t_drop"] = float(e.get("t"))
            passengers[rid]["d"] = (e["lon"], e["lat"])

    individual_detours = []
    for rid, p in passengers.items():
        if "t_drop" not in p: continue
        
        # 분자: 실제 탑승 시간(Ride Time) = 하차 시각 - 승차 시각
        actual_ride_time = p["t_drop"] - p["t_pick"]
        
        # 분모: 최단 도로 주행 시간(Shortest Duration)
        shortest_duration = osrm_route_duration_sec(p["o"][0], p["o"][1], p["d"][0], p["d"][1])
        
        if shortest_duration > 5: # 매우 짧은 거리는 수치 왜곡 방지를 위해 5초 이상만 계산
            ratio = actual_ride_time / shortest_duration
            # 우회율은 이론상 1.0 미만일 수 없으므로(도로망 최단시간 기준) 최소값 1.0 보정
            individual_detours.append(max(1.0, ratio))
            
    avg_detour_ratio = round(sum(individual_detours) / len(individual_detours), 3) if individual_detours else 0.0

    # 결과 취합
    rows.append({
        "시나리오": scen,
        "배차성공인원": num_passengers,
        "배차성공율(%)": round(num_passengers / len(req_df) * 100, 2),
        "미수행 요청": len(summary.get("rejected", [])),
        "평균 대기시간(초)": round(merged["wait_sec"].mean(), 1) if not merged.empty else 0,
        "최대 대기시간(초)": round(merged["wait_sec"].max(), 1) if not merged.empty else 0,
        "우회율": avg_detour_ratio
    })

# --------- ③ 최종 결과 출력 ---------
result_df = pd.DataFrame(rows).sort_values("시나리오").reset_index(drop=True)
result_df

,시나리오,배차성공인원,배차성공율(%),미수행 요청,평균 대기시간(초),최대 대기시간(초),우회율
0,10대(nonkey1),495,36.24,871,394.5,1233.0,2.276
1,20대(nonkey1),813,59.52,553,330.0,1087.0,2.319
2,30대_nonkey1,1046,76.57,320,300.0,1218.0,2.345
3,50대_nonkey1,1319,96.56,47,201.0,1151.0,1.975


### 차량 대수별 성능평가

- 기본전제 : 배차주기 (60초) / 우회율 / 대기시간 / 차량정원 / 삽입창 개수 등 모두 동일
- 오로지 차량대수만 다름

#### 테스트 버전 확인

In [3]:
# ==== 시나리오 성능 비교 (OSRM 기반 우회율 + 서비스 지표 집중 버전) ====
import os, json, time, math
from pathlib import Path
import pandas as pd
import requests
from collections import defaultdict

# --------- 사용자 입력 (상대 경로 반영) ---------
PARQUET_PATH = "../data/processed/420_540_trip_both.parquet"
OSRM_BASE = "http://127.0.0.1:8000"  # OSRM 포트 8000 반영

SCENARIOS = {
    "10대": "../outputs/10대",
    "20대": "../outputs/20대",
    "30대": "../outputs/30대",
    "30대_test": "../outputs/30대_test",
    "50대": "../outputs/50대",
    "50대_test": "../outputs/50대_test",
}

# --------- 유틸 함수 ---------
def jload(p):
    if not os.path.exists(p): return None
    with open(p, "r", encoding="utf-8") as f:
        try: return json.load(f)
        except:
            f.seek(0); return [json.loads(line) for line in f if line.strip()]

def get_list(x):
    if isinstance(x, list): return x
    if isinstance(x, dict):
        for k in ("events","moves"):
            if isinstance(x.get(k), list): return x[k]
    return []

_OSRM_CACHE = {}
def osrm_route_km(lon1, lat1, lon2, lat2):
    if None in (lon1, lat1, lon2, lat2): return 0.0
    key = (round(float(lon1),6), round(float(lat1),6), round(float(lon2),6), round(float(lat2),6))
    if key in _OSRM_CACHE: return _OSRM_CACHE[key]
    url = f"{OSRM_BASE}/route/v1/driving/{lon1},{lat1};{lon2},{lat2}"
    try:
        r = requests.get(url, params={"overview":"false"}, timeout=2.0)
        if r.status_code == 200:
            km = r.json()["routes"][0]["distance"] / 1000.0
            _OSRM_CACHE[key] = km
            return km
    except: pass
    return 0.0 # 실패 시 0

def haversine_km(lon1, lat1, lon2, lat2):
    R = 6371.0
    dlon, dlat = math.radians(lon2-lon1), math.radians(lat2-lat1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2
    return 2*R*math.asin(math.sqrt(max(0,a)))

# --------- ① 데이터 로드 ---------
pq = pd.read_parquet(PARQUET_PATH).copy()
pq["KEY1"] = pq["KEY1"].astype(str)
req_df = pd.DataFrame({
    "req_id": pq["KEY1"],
    "t_request": pd.to_numeric(pq["승차_timestamp"], errors="coerce"),
    "o_lon": pd.to_numeric(pq["출발_x"], errors="coerce"), "o_lat": pd.to_numeric(pq["출발_y"], errors="coerce"),
    "d_lon": pd.to_numeric(pq["도착_x"], errors="coerce"), "d_lat": pd.to_numeric(pq["도착_y"], errors="coerce")
})
req_df = req_df.dropna(subset=["t_request"])

# --------- ② 분석 루프 ---------
rows = []
for scen, folder in SCENARIOS.items():
    if not os.path.exists(folder): continue
    events = get_list(jload(os.path.join(folder, "events.json")))
    moves = get_list(jload(os.path.join(folder, "moves.json")))
    summary = jload(os.path.join(folder, "summary.json")) or {}

    # 1) 지표 계산 (Served/Wait)
    pick_times = {str(e["req_id"]): float(e["t"]) for e in events if str(e.get("type")).upper()=="PICKUP"}
    num_passengers = len(pick_times)
    
    pick_df = pd.DataFrame({"req_id": list(pick_times.keys()), "t_pickup": list(pick_times.values())})
    merged = req_df.merge(pick_df, on="req_id")
    merged["wait_sec"] = (merged["t_pickup"] - merged["t_request"]).clip(lower=0)
    
    # 2) 우회율 계산 (OSRM 기반)
    with_pax_km_osrm = 0.0
    for mv in moves:
        if float(mv.get("load", 0)) > 0:
            with_pax_km_osrm += osrm_route_km(mv["lon1"], mv["lat1"], mv["lon2"], mv["lat2"])
    
    served_ids = set(summary.get("served", [])) | set(pick_times.keys())
    served_reqs = req_df[req_df["req_id"].isin(served_ids)]
    straight_km_sum = sum(osrm_route_km(r.o_lon, r.o_lat, r.d_lon, r.d_lat) for r in served_reqs.itertuples())
    detour_ratio = round(with_pax_km_osrm / straight_km_sum, 3) if straight_km_sum > 0 else 0.0

    rows.append({
        "시나리오": scen,
        "배차성공인원": num_passengers,
        "배차성공율(%)": round(num_passengers / len(req_df) * 100, 2),
        "미수행 요청": len(summary.get("rejected", [])),
        "평균 대기시간(초)": round(merged["wait_sec"].mean(), 1) if not merged.empty else 0,
        "최대 대기시간(초)": round(merged["wait_sec"].max(), 1) if not merged.empty else 0,
        "우회율": detour_ratio,
        "유차거리(km)": round(with_pax_km_osrm, 1)
    })

# --------- ③ 결과 출력 ---------
pd.DataFrame(rows).sort_values("시나리오").reset_index(drop=True)

,시나리오,배차성공인원,배차성공율(%),미수행 요청,평균 대기시간(초),최대 대기시간(초),우회율,유차거리(km)
0,10대,488,35.72,878,393.4,1151.0,0.905,783.3
1,20대,858,62.81,508,337.6,1214.0,0.884,1453.4
2,30대,1087,79.58,279,300.4,1123.0,0.912,2006.0
3,30대_test,1020,74.67,346,330.0,1214.0,0.984,2007.1
4,50대,1359,99.49,7,198.8,1231.0,0.885,2560.2
5,50대_test,1313,96.12,53,236.6,1146.0,0.532,1481.4


#### osrm기반 우회율 계산 후 전체 성능평가 지표 분석 (기존거)

In [ ]:
# ==== 시나리오 성능 비교 (OSRM 기반 우회율 버전) ====
# - 평균/최대 대기시간(초): parquet(승차_timestamp) ↔ events(PICKUP) 매칭
# - 배차성공율(%): 배차성공인원 / 총요청수 * 100
# - 우회율: 개별 승객별 (실제거리/최단거리) 비율의 평균
# - 차량 적재율(%): 유차거리(하버사인) / 총주행거리(하버사인) * 100
# - 공차비율(%): 공차거리 / 총주행거리 * 100
# - 운행 효율성(%): 적재율 × 배차성공율 / 100

import os, json, time, math
from pathlib import Path
import pandas as pd
import requests
from collections import defaultdict

# --------- 사용자 입력 ---------
PARQUET_PATH = r"C:\dynamic_wirye_drt\data\parquet\420_540_trip_both.parquet"

SCENARIOS = {
    "10대": r"C:\dynamic_wirye_drt\outputs\10대",
    "20대": r"C:\dynamic_wirye_drt\outputs\20대",
    "30대": r"C:\dynamic_wirye_drt\outputs\30대",
    "50대": r"C:\dynamic_wirye_drt\outputs\50대",
}

# (옵션) CSV 저장 경로
# OUT_CSV = r"C:\dynamic_wirye_drt\outputs\scenario_metrics.csv"

# --------- 공통 유틸 ---------
def jload(p):
    if not os.path.exists(p):
        return None
    with open(p, "r", encoding="utf-8") as f:
        try:
            return json.load(f)
        except Exception:
            f.seek(0)
            return [json.loads(line) for line in f if line.strip()]

def get_list(x):
    if x is None: return []
    if isinstance(x, list): return x
    if isinstance(x, dict):
        for k in ("events","moves","points","data","rows","list"):
            if isinstance(x.get(k), list):
                return x[k]
    return []

def to_seconds_series(sr):
    sr = pd.to_numeric(sr, errors="coerce")
    med = sr.dropna().median()
    if pd.isna(med): return sr
    if med < 1_000:     # 분 → 초
        return sr * 60.0
    if med > 1e11:      # ms → s
        return sr / 1000.0
    return sr           # 이미 초

def haversine_km(lon1, lat1, lon2, lat2):
    R = 6371.0
    dlon = math.radians((lon2 - lon1) if pd.notna(lon2) and pd.notna(lon1) else 0.0)
    dlat = math.radians((lat2 - lat1) if pd.notna(lat2) and pd.notna(lat1) else 0.0)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2
    return 2*R*math.asin(math.sqrt(max(0.0, a)))

# --------- OSRM 거리 유틸(캐시 포함) ---------
OSRM_BASE = "http://127.0.0.1:5000"  # osrm-routed 주소
_OSRM_CACHE = {}

def _cache_key(a, b):
    # 좌표를 소수점 6자리로 반올림(캐시 키 안정화)
    lon1, lat1 = round(float(a[0]), 6), round(float(a[1]), 6)
    lon2, lat2 = round(float(b[0]), 6), round(float(b[1]), 6)
    return (lon1, lat1, lon2, lat2)

def osrm_route_km(lon1, lat1, lon2, lat2, timeout=3.0):
    """두 점 사이 도로거리(km). 실패 시 하버사인 km로 폴백."""
    if None in (lon1, lat1, lon2, lat2):
        return 0.0
    key = _cache_key((lon1,lat1), (lon2,lat2))
    if key in _OSRM_CACHE:
        return _OSRM_CACHE[key]

    url = f"{OSRM_BASE}/route/v1/driving/{lon1},{lat1};{lon2},{lat2}"
    try:
        r = requests.get(url, params={"overview":"false"}, timeout=timeout)
        if r.status_code == 200:
            js = r.json()
            if js.get("code") == "Ok" and js.get("routes"):
                km = js["routes"][0]["distance"] / 1000.0
                _OSRM_CACHE[key] = km
                return km
    except Exception:
        pass

    # 폴백: 하버사인
    km = haversine_km(lon1, lat1, lon2, lat2)
    _OSRM_CACHE[key] = km
    return km

# --------- 원본 파케: 요청시각/OD 좌표 ---------
pq = pd.read_parquet(PARQUET_PATH).copy()
pq["KEY1"] = pq["KEY1"].astype(str)
req_df = pd.DataFrame({
    "req_id": pq["KEY1"].astype(str),
    "t_request": to_seconds_series(pq["승차_timestamp"])
})
# OD 좌표(우회율 계산에 사용)
for c_from, c_to in [("출발_x","o_lon"),("출발_y","o_lat"),("도착_x","d_lon"),("도착_y","d_lat")]:
    if c_from in pq.columns:
        req_df[c_to] = pd.to_numeric(pq[c_from], errors="coerce")
req_df = req_df.dropna(subset=["t_request"])
requests_total = len(req_df)

print(f"📊 총 요청 수: {requests_total}")
print(f"📊 req_df 컬럼: {list(req_df.columns)}")

# --------- 시나리오 루프 ---------
rows = []
for scen, folder in SCENARIOS.items():
    t0 = time.perf_counter()
    print(f"\n🔄 처리 중: {scen}")

    ev_path = str(Path(folder)/"events.json")
    mv_path = str(Path(folder)/"moves.json")
    sum_path= str(Path(folder)/"summary.json")

    events = get_list(jload(ev_path))
    moves  = get_list(jload(mv_path))
    summary= jload(sum_path) or {}

    print(f"📁 {scen}: events={len(events)}, moves={len(moves)}")

    # --- 이용객수(=PICKUP 수) ---
    pick_times = {}
    for e in events:
        if str(e.get("type","")).upper()!="PICKUP":
            continue
        rid = str(e.get("req_id"))
        tt  = e.get("t") or e.get("ts") or e.get("timestamp") or e.get("time")
        if rid is None or tt is None:
            continue
        tt = float(tt)
        if (rid not in pick_times) or (tt < pick_times[rid]):
            pick_times[rid] = tt
    num_passengers = len(pick_times)

    print(f"👥 {scen}: PICKUP 이벤트 {num_passengers}개")

    # --- 미수행 요청 ---
    if isinstance(summary, dict) and isinstance(summary.get("rejected"), list):
        unserved = len(summary["rejected"])
    else:
        # fallback: 총요청 - 픽업
        unserved = max(0, requests_total - num_passengers)

    # --- 대기시간 (요청↔PICKUP 매칭, 조기픽업은 0으로 클립) ---
    pick_df = pd.DataFrame({"req_id": list(pick_times.keys()), "t_pickup": list(pick_times.values())})
    merged = req_df.merge(pick_df, on="req_id", how="inner")
    merged["wait_sec"] = (merged["t_pickup"] - merged["t_request"]).clip(lower=0)
    avg_wait = float(merged["wait_sec"].mean()) if len(merged) else 0.0
    max_wait = float(merged["wait_sec"].max()) if len(merged) else 0.0

    # --- 주행거리 집계 ---
    total_km = 0.0
    with_pax_km_haversine = 0.0
    with_pax_km_osrm = 0.0

    for mv in moves:
        lon1, lat1 = mv.get("lon1"), mv.get("lat1")
        lon2, lat2 = mv.get("lon2"), mv.get("lat2")
        if None in (lon1,lat1,lon2,lat2):
            continue

        d_hav = haversine_km(float(lon1), float(lat1), float(lon2), float(lat2))
        total_km += d_hav

        if mv.get("load", 0) and float(mv.get("load", 0)) > 0:
            with_pax_km_haversine += d_hav
            d_osrm = osrm_route_km(float(lon1), float(lat1), float(lon2), float(lat2))
            with_pax_km_osrm += d_osrm

    deadhead_km   = max(0.0, total_km - with_pax_km_haversine)
    deadhead_ratio = (deadhead_km / total_km * 100.0) if total_km > 0 else 0.0
    occupancy_pct  = (with_pax_km_haversine / total_km * 100.0) if total_km > 0 else 0.0

    # --- 우회율(%): 개별 승객별 계산 (수정된 버전) ---
    detour_ratios = []
    
    # PICKUP/DROPOFF 이벤트에서 좌표 추출
    pickup_coords = {}
    dropoff_coords = {}
    
    for e in events:
        event_type = str(e.get("type", "")).upper()
        req_id = str(e.get("req_id", ""))
        if not req_id:
            continue
            
        # 좌표 추출 (여러 가능한 필드명 시도)
        lon = e.get('lon') or e.get('longitude') or e.get('x')
        lat = e.get('lat') or e.get('latitude') or e.get('y')
        
        if event_type == "PICKUP" and lon is not None and lat is not None:
            pickup_coords[req_id] = (float(lon), float(lat))
        elif event_type == "DROPOFF" and lon is not None and lat is not None:
            dropoff_coords[req_id] = (float(lon), float(lat))
    
    print(f"🚗 {scen}: PICKUP 좌표 {len(pickup_coords)}개, DROPOFF 좌표 {len(dropoff_coords)}개")
    
    # 좌표가 있는 경우에만 우회율 계산
    if pickup_coords and dropoff_coords:
        # 공통 req_id 찾기
        common_req_ids = set(pickup_coords.keys()) & set(dropoff_coords.keys())
        print(f"🔗 {scen}: PICKUP-DROPOFF 매칭된 요청 {len(common_req_ids)}개")
        
        for req_id in common_req_ids:
            pickup_lon, pickup_lat = pickup_coords[req_id]
            dropoff_lon, dropoff_lat = dropoff_coords[req_id]
            
            # 직선 최단거리 계산
            direct_distance_km = osrm_route_km(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
            
            if direct_distance_km > 0.1:  # 최소 100m 이상
                # 실제 거리는 평균값으로 근사 (개별 추적이 어려운 경우)
                if len(common_req_ids) > 0:
                    avg_actual_per_passenger = with_pax_km_osrm / len(common_req_ids) if with_pax_km_osrm > 0 else with_pax_km_haversine / len(common_req_ids)
                    
                    if avg_actual_per_passenger > 0:
                        ratio = avg_actual_per_passenger / direct_distance_km
                        detour_ratios.append(ratio)
    
    # 우회율이 계산되지 않은 경우 원본 파케 좌표 사용
    if not detour_ratios and {"o_lon","o_lat","d_lon","d_lat"}.issubset(req_df.columns):
        print(f"🔄 {scen}: events에서 우회율 계산 실패, req_df 좌표 사용")
        
        served_ids = set(pick_times.keys())  # PICKUP된 req_id들
        served_reqs = req_df[req_df["req_id"].isin(served_ids)]
        
        baseline_km_sum = 0.0
        valid_count = 0
        
        for _, r in served_reqs.iterrows():
            olon, olat, dlon, dlat = r["o_lon"], r["o_lat"], r["d_lon"], r["d_lat"]
            if pd.notna(olon) and pd.notna(olat) and pd.notna(dlon) and pd.notna(dlat):
                direct_km = osrm_route_km(float(olon), float(olat), float(dlon), float(dlat))
                if direct_km > 0.1:
                    baseline_km_sum += direct_km
                    valid_count += 1
        
        if baseline_km_sum > 0 and with_pax_km_osrm > 0:
            avg_ratio = with_pax_km_osrm / baseline_km_sum
            detour_ratios = [avg_ratio]  # 전체 평균을 하나의 비율로
            print(f"📐 {scen}: 전체 평균 우회율 사용 (ratio={avg_ratio:.3f})")
        elif baseline_km_sum > 0 and with_pax_km_haversine > 0:
            avg_ratio = with_pax_km_haversine / baseline_km_sum
            detour_ratios = [avg_ratio]
            print(f"📐 {scen}: 하버사인 평균 우회율 사용 (ratio={avg_ratio:.3f})")
    
    # 최종 우회율 계산 (비율로 유지)
    if detour_ratios:
        detour_ratio = sum(detour_ratios) / len(detour_ratios)
        print(f"✅ {scen}: 우회율 {detour_ratio:.3f} (샘플 {len(detour_ratios)}개)")
        
        # 극값 필터링 (경고만)
        if detour_ratio < 0.5:
            print(f"⚠️ {scen}: 낮은 우회율 ({detour_ratio:.3f}) - 매우 효율적")
        elif detour_ratio > 4.0:
            print(f"⚠️ {scen}: 높은 우회율 ({detour_ratio:.3f}) - 매우 비효율적")
    else:
        detour_ratio = 0.0
        print(f"❌ {scen}: 우회율 계산 불가")

    # --- 배차성공율(%) ---
    service_rate_pct = (num_passengers / requests_total * 100.0) if requests_total>0 else 0.0

    # --- 운행 효율성(%): 적재율 × 배차성공율 ---
    operational_eff = occupancy_pct * service_rate_pct / 100.0

    rows.append({
        "시나리오": scen,
        "배차성공인원": int(num_passengers),
        "배차성공율(%)": round(service_rate_pct, 2),
        "미수행 요청": int(unserved),
        "평균 대기시간(초)": round(avg_wait, 2),
        "최대 대기시간(초)": round(max_wait, 2),
        "우회율": round(detour_ratio, 3),
        "차량 적재율(%)": round(occupancy_pct, 2),
        "총주행거리(km)": round(total_km, 2),
        "유차거리(km)": round(with_pax_km_haversine, 2),
        "공차거리(km)": round(deadhead_km, 2),
        "공차비율(%)": round(deadhead_ratio, 2),
        "운행 효율성(%)": round(operational_eff, 2),
    })

    print(f"⏱️ {scen} 완료: {time.perf_counter() - t0:.1f}초")

# --------- 결과표 ---------
print(f"\n🎯 전체 처리 완료!")
df = pd.DataFrame(rows).sort_values("시나리오").reset_index(drop=True)
df

# (옵션) CSV 저장
# df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")

📊 총 요청 수: 1366
📊 req_df 컬럼: ['req_id', 't_request', 'o_lon', 'o_lat', 'd_lon', 'd_lat']

🔄 처리 중: 10대
📁 10대: events=2342, moves=2083
👥 10대: PICKUP 이벤트 488개
🚗 10대: PICKUP 좌표 488개, DROPOFF 좌표 488개
🔗 10대: PICKUP-DROPOFF 매칭된 요청 488개
✅ 10대: 우회율 1.236 (샘플 487개)
⏱️ 10대 완료: 24.3초

🔄 처리 중: 20대
📁 20대: events=3082, moves=3789
👥 20대: PICKUP 이벤트 858개
🚗 20대: PICKUP 좌표 858개, DROPOFF 좌표 858개
🔗 20대: PICKUP-DROPOFF 매칭된 요청 858개
✅ 20대: 우회율 1.153 (샘플 857개)
⏱️ 20대 완료: 33.6초

🔄 처리 중: 20대_다시비교
📁 20대_다시비교: events=3082, moves=3789
👥 20대_다시비교: PICKUP 이벤트 858개
🚗 20대_다시비교: PICKUP 좌표 858개, DROPOFF 좌표 858개
🔗 20대_다시비교: PICKUP-DROPOFF 매칭된 요청 858개
✅ 20대_다시비교: 우회율 1.153 (샘플 857개)
⏱️ 20대_다시비교 완료: 0.0초

🔄 처리 중: 20대_제약조건조정
📁 20대_제약조건조정: events=2932, moves=3613
👥 20대_제약조건조정: PICKUP 이벤트 783개
🚗 20대_제약조건조정: PICKUP 좌표 783개, DROPOFF 좌표 783개
🔗 20대_제약조건조정: PICKUP-DROPOFF 매칭된 요청 783개
✅ 20대_제약조건조정: 우회율 1.221 (샘플 782개)
⏱️ 20대_제약조건조정 완료: 27.4초

🔄 처리 중: 20대_대기15분
📁 20대_대기15분: events=3122, moves=3895
👥 20대_대기15분: PICKUP 이벤트 878개
🚗 20대_대기

,시나리오,배차성공인원,배차성공율(%),미수행 요청,평균 대기시간(초),최대 대기시간(초),우회율,차량 적재율(%),총주행거리(km),유차거리(km),공차거리(km),공차비율(%),운행 효율성(%)
0,10대,488,35.72,878,393.39,1151.0,1.236,85.46,621.44,531.10,90.34,14.54,30.53
1,20대,858,62.81,508,337.61,1214.0,1.153,83.44,1159.77,967.75,192.01,16.56,52.41
2,20대_다시비교,858,62.81,508,337.61,1214.0,1.153,83.44,1159.77,967.75,192.01,16.56,52.41
3,20대_대기15분,878,64.28,488,443.42,1385.0,1.149,84.86,1185.54,1006.10,179.44,15.14,54.55
4,20대_제약조건조정,783,57.32,583,350.81,1255.0,1.221,81.95,1161.55,951.86,209.69,18.05,46.97
5,30대,1087,79.58,279,300.42,1123.0,1.177,83.09,1684.60,1399.72,284.87,16.91,66.12
6,50대,1359,99.49,7,198.75,1231.0,1.122,78.14,2499.03,1952.78,546.24,21.86,77.74


#### 위 코드에서 지표 일부 삭제

In [11]:
# ==== 시나리오 성능 비교 (OSRM 기반 우회율 + 서비스 지표 집중 버전) ====
import os, json, time, math
from pathlib import Path
import pandas as pd
import requests

# --------- 사용자 입력 (상대 경로 및 포트 8000 반영) ---------
PARQUET_PATH = "../data/processed/420_540_trip_both_sequential.parquet"
OSRM_BASE = "http://127.0.0.1:8000" 

SCENARIOS = {
    # "10대": "../outputs/10대",
    # "20대": "../outputs/20대",
    "30대": "../outputs/30대(nonkey1)",
    "50대": "../outputs/50대(nonkey1)",
}

# --------- 유틸 함수 ---------
def jload(p):
    if not os.path.exists(p): return None
    with open(p, "r", encoding="utf-8") as f:
        try: return json.load(f)
        except:
            f.seek(0); return [json.loads(line) for line in f if line.strip()]

def get_list(x):
    if isinstance(x, list): return x
    if isinstance(x, dict):
        for k in ("events","moves"):
            if isinstance(x.get(k), list): return x[k]
    return []

_OSRM_CACHE = {}
def osrm_route_km(lon1, lat1, lon2, lat2):
    if None in (lon1, lat1, lon2, lat2): return 0.0
    key = (round(float(lon1),6), round(float(lat1),6), round(float(lon2),6), round(float(lat2),6))
    if key in _OSRM_CACHE: return _OSRM_CACHE[key]
    url = f"{OSRM_BASE}/route/v1/driving/{lon1},{lat1};{lon2},{lat2}"
    try:
        r = requests.get(url, params={"overview":"false"}, timeout=2.0)
        if r.status_code == 200:
            km = r.json()["routes"][0]["distance"] / 1000.0
            _OSRM_CACHE[key] = km
            return km
    except: pass
    return 0.0 

# --------- ① 데이터 로드 ---------
pq = pd.read_parquet(PARQUET_PATH).copy()
pq["KEY1"] = pq["KEY1"].astype(str)
req_df = pd.DataFrame({
    "req_id": pq["KEY1"],
    "t_request": pd.to_numeric(pq["승차_timestamp"], errors="coerce"),
    "o_lon": pd.to_numeric(pq["출발_x"], errors="coerce"), "o_lat": pd.to_numeric(pq["출발_y"], errors="coerce"),
    "d_lon": pd.to_numeric(pq["도착_x"], errors="coerce"), "d_lat": pd.to_numeric(pq["도착_y"], errors="coerce")
})
req_df = req_df.dropna(subset=["t_request"])

# --------- ② 분석 루프 ---------
rows = []
for scen, folder in SCENARIOS.items():
    if not os.path.exists(folder): continue
    events = get_list(jload(os.path.join(folder, "events.json")))
    moves = get_list(jload(os.path.join(folder, "moves.json")))
    summary = jload(os.path.join(folder, "summary.json")) or {}

    # 1) 지표 계산 (Served/Wait)
    pick_times = {str(e["req_id"]): float(e["t"]) for e in events if str(e.get("type")).upper()=="PICKUP"}
    num_passengers = len(pick_times)
    
    pick_df = pd.DataFrame({"req_id": list(pick_times.keys()), "t_pickup": list(pick_times.values())})
    merged = req_df.merge(pick_df, on="req_id")
    merged["wait_sec"] = (merged["t_pickup"] - merged["t_request"]).clip(lower=0)
    
    # 2) 우회율 계산 (실제 유차 주행거리 / 승객별 최단거리 합계)
    with_pax_km_osrm = 0.0
    for mv in moves:
        # 차량에 승객이 1명이라도 타고 있을 때(load > 0)의 주행거리 합산
        if float(mv.get("load", 0)) > 0:
            with_pax_km_osrm += osrm_route_km(mv["lon1"], mv["lat1"], mv["lon2"], mv["lat2"])
    
    # 배차 성공한 승객들의 '최단 도로 거리' 합계 구하기
    served_ids = set(summary.get("served", [])) | set(pick_times.keys())
    served_reqs = req_df[req_df["req_id"].isin(served_ids)]
    straight_km_sum = sum(osrm_route_km(r.o_lon, r.o_lat, r.d_lon, r.d_lat) for r in served_reqs.itertuples())
    
    detour_ratio = round(with_pax_km_osrm / straight_km_sum, 3) if straight_km_sum > 0 else 0.0

    rows.append({
        "시나리오": scen,
        "배차성공인원": num_passengers,
        "배차성공율(%)": round(num_passengers / len(req_df) * 100, 2),
        "미수행 요청": len(summary.get("rejected", [])),
        "평균 대기시간(초)": round(merged["wait_sec"].mean(), 1) if not merged.empty else 0,
        "최대 대기시간(초)": round(merged["wait_sec"].max(), 1) if not merged.empty else 0,
        "우회율": detour_ratio
    })

# --------- ③ 결과 출력 ---------
pd.DataFrame(rows).sort_values("시나리오").reset_index(drop=True)

,시나리오,배차성공인원,배차성공율(%),미수행 요청,평균 대기시간(초),최대 대기시간(초),우회율
0,30대,1046,76.57,320,300.0,1218.0,0.938
1,50대,1319,96.56,47,201.0,1151.0,0.924


In [12]:
# ==== 시나리오 성능 비교 (OSRM 기반 우회율 + 서비스 지표 집중 버전) ====
import os, json, time, math
from pathlib import Path
import pandas as pd
import requests

# --------- 사용자 입력 (상대 경로 및 포트 8000 반영) ---------
PARQUET_PATH = "../data/processed/420_540_trip_both.parquet"
OSRM_BASE = "http://127.0.0.1:8000" 

SCENARIOS = {
    # "10대": "../outputs/10대",
    # "20대": "../outputs/20대",
    "30대": "../outputs/30대",
    "50대": "../outputs/50대",
}

# --------- 유틸 함수 ---------
def jload(p):
    if not os.path.exists(p): return None
    with open(p, "r", encoding="utf-8") as f:
        try: return json.load(f)
        except:
            f.seek(0); return [json.loads(line) for line in f if line.strip()]

def get_list(x):
    if isinstance(x, list): return x
    if isinstance(x, dict):
        for k in ("events","moves"):
            if isinstance(x.get(k), list): return x[k]
    return []

_OSRM_CACHE = {}
def osrm_route_km(lon1, lat1, lon2, lat2):
    if None in (lon1, lat1, lon2, lat2): return 0.0
    key = (round(float(lon1),6), round(float(lat1),6), round(float(lon2),6), round(float(lat2),6))
    if key in _OSRM_CACHE: return _OSRM_CACHE[key]
    url = f"{OSRM_BASE}/route/v1/driving/{lon1},{lat1};{lon2},{lat2}"
    try:
        r = requests.get(url, params={"overview":"false"}, timeout=2.0)
        if r.status_code == 200:
            km = r.json()["routes"][0]["distance"] / 1000.0
            _OSRM_CACHE[key] = km
            return km
    except: pass
    return 0.0 

# --------- ① 데이터 로드 ---------
pq = pd.read_parquet(PARQUET_PATH).copy()
pq["KEY1"] = pq["KEY1"].astype(str)
req_df = pd.DataFrame({
    "req_id": pq["KEY1"],
    "t_request": pd.to_numeric(pq["승차_timestamp"], errors="coerce"),
    "o_lon": pd.to_numeric(pq["출발_x"], errors="coerce"), "o_lat": pd.to_numeric(pq["출발_y"], errors="coerce"),
    "d_lon": pd.to_numeric(pq["도착_x"], errors="coerce"), "d_lat": pd.to_numeric(pq["도착_y"], errors="coerce")
})
req_df = req_df.dropna(subset=["t_request"])

# --------- ② 분석 루프 ---------
rows = []
for scen, folder in SCENARIOS.items():
    if not os.path.exists(folder): continue
    events = get_list(jload(os.path.join(folder, "events.json")))
    moves = get_list(jload(os.path.join(folder, "moves.json")))
    summary = jload(os.path.join(folder, "summary.json")) or {}

    # 1) 지표 계산 (Served/Wait)
    pick_times = {str(e["req_id"]): float(e["t"]) for e in events if str(e.get("type")).upper()=="PICKUP"}
    num_passengers = len(pick_times)
    
    pick_df = pd.DataFrame({"req_id": list(pick_times.keys()), "t_pickup": list(pick_times.values())})
    merged = req_df.merge(pick_df, on="req_id")
    merged["wait_sec"] = (merged["t_pickup"] - merged["t_request"]).clip(lower=0)
    
    # 2) 우회율 계산 (실제 유차 주행거리 / 승객별 최단거리 합계)
    with_pax_km_osrm = 0.0
    for mv in moves:
        # 차량에 승객이 1명이라도 타고 있을 때(load > 0)의 주행거리 합산
        if float(mv.get("load", 0)) > 0:
            with_pax_km_osrm += osrm_route_km(mv["lon1"], mv["lat1"], mv["lon2"], mv["lat2"])
    
    # 배차 성공한 승객들의 '최단 도로 거리' 합계 구하기
    served_ids = set(summary.get("served", [])) | set(pick_times.keys())
    served_reqs = req_df[req_df["req_id"].isin(served_ids)]
    straight_km_sum = sum(osrm_route_km(r.o_lon, r.o_lat, r.d_lon, r.d_lat) for r in served_reqs.itertuples())
    
    detour_ratio = round(with_pax_km_osrm / straight_km_sum, 3) if straight_km_sum > 0 else 0.0

    rows.append({
        "시나리오": scen,
        "배차성공인원": num_passengers,
        "배차성공율(%)": round(num_passengers / len(req_df) * 100, 2),
        "미수행 요청": len(summary.get("rejected", [])),
        "평균 대기시간(초)": round(merged["wait_sec"].mean(), 1) if not merged.empty else 0,
        "최대 대기시간(초)": round(merged["wait_sec"].max(), 1) if not merged.empty else 0,
        "우회율": detour_ratio
    })

# --------- ③ 결과 출력 ---------
pd.DataFrame(rows).sort_values("시나리오").reset_index(drop=True)

,시나리오,배차성공인원,배차성공율(%),미수행 요청,평균 대기시간(초),최대 대기시간(초),우회율
0,30대,1087,79.58,279,300.4,1123.0,0.912
1,50대,1359,99.49,7,198.8,1231.0,0.885
